In [1]:
import json
import os
import shutil
import random

### Converting JSON to YOLO input format

In [2]:
def convert_to_yolo_format(json_data, img_width, img_height):
    """
    Function to convert json labels data to yolo format
    
    Args:
        json_data (dict): json data from X-anylabelling
        img_width (int): width of the image
        img_height (int): height of the image
    """
    yolo_format = []

    for shape in json_data["shapes"]:
        points = shape["points"]
        
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]

        xmin = min(x_coords)
        xmax = max(x_coords)
        ymin = min(y_coords)
        ymax = max(y_coords)

        x_center = (xmin + xmax) / 2
        y_center = (ymin + ymax) / 2
        width = xmax - xmin
        height = ymax - ymin

        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        class_id = 0

        yolo_format.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_format

In [1]:
def process_json_file(json_file_path, output_dir):
    """
    Function to process a json file and convert it to yolo format

    Args:
        json_file_path (str): path to the json file
        output_dir (str): output directory to save the yolo format file
    """
    with open(json_file_path, 'r') as f:
        json_data = json.load(f)
    
    img_width = json_data["imageWidth"]
    img_height = json_data["imageHeight"]

    yolo_labels = convert_to_yolo_format(json_data, img_width, img_height)

    image_name = os.path.splitext(os.path.basename(json_file_path))[0]
    output_file_path = os.path.join(output_dir, f"{image_name}.txt")

    with open(output_file_path, 'w') as f:
        for label in yolo_labels:
            f.write(f"{label}\n")

In [4]:
def process_json_directory(json_dir, output_dir):
    """
    Function to process a directory of json files and convert them to yolo format
    
    Args:
        json_dir (str): directory containing json files
        output_dir (str): output directory to save the yolo format files
    """
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for json_file in os.listdir(json_dir):
        if json_file.endswith(".json"):
            json_file_path = os.path.join(json_dir, json_file)
            process_json_file(json_file_path, output_dir)

In [5]:
if __name__ == "__main__":
    json_dir = "/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/json_files"
    
    output_dir = "/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/labels"
    
    process_json_directory(json_dir, output_dir)

### Splitting Data into Training and Validation Sets

In [ ]:
images_folder = '/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/images'
labels_folder = '/home/chaitanya/Academics/3rd)Semester/DLRV/project/data/labels'
train_images_folder = '/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/images/train'
train_labels_folder = '/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/labels/train'
val_images_folder = '/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/images/val'
val_labels_folder = '/home/chaitanya/Academics/3rd_Semester/DLRV/project/data/labels/val'

os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)

image_files = [f for f in os.listdir(images_folder) if f.endswith(('.png', '.jpg'))]

random.shuffle(image_files)

train_images = image_files[:1000]
val_images = image_files[1000:1200]

for image in train_images:
    image_path = os.path.join(images_folder, image)
    label_name = os.path.splitext(image)[0] + '.txt'
    label_path = os.path.join(labels_folder, label_name)

    shutil.move(image_path, os.path.join(train_images_folder, image))

    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(train_labels_folder, label_name))

for image in val_images:
    image_path = os.path.join(images_folder, image)
    label_name = os.path.splitext(image)[0] + '.txt'
    label_path = os.path.join(labels_folder, label_name)

    shutil.move(image_path, os.path.join(val_images_folder, image))

    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(val_labels_folder, label_name))

print("Data successfully split into train and validation sets.")